In [1]:
%reload_ext autoreload
%autoreload 2

# Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import tqdm.notebook as tqdm

In [3]:
from polyloss import PolyLoss, PolyFocalLoss
from polynloss import polynloss, polynfocal

# Helper Functions

In [4]:
# seed function
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
class MyModel(nn.Module):
    def __init__(self, in_ch=784, hidden_ch=500, out_ch=10):
        super().__init__()
        self.l1 = nn.Linear(in_ch, hidden_ch, bias=True)
        self.fc = nn.Linear(hidden_ch, out_ch, bias=True)
        
    def forward(self, x):
        out = self.l1(x)
        out = F.relu(out)
        return self.fc(out)

In [6]:
# track the metrices
class AvgStats(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.losses =[]
        self.accs =[]
        self.f1 = []
        self.precs = []
        self.recs = []
        self.tprs = []
        self.fprs = []
        self.its = []
        
    def append(self, loss, acc, f1, prec, rec, it):
        self.losses.append(loss)
        self.accs.append(acc)
        self.f1.append(f1)
        self.precs.append(prec)
        self.recs.append(rec)
        self.its.append(it)

In [7]:
# Save model
def save_checkpoint(model, fp):
    """Save checkpoint if a new best is achieved"""
    torch.save(model.state_dict(), fp)

In [8]:
# load model
def load_checkpoint(model, fp):
    sd = torch.load(fp, map_location=lambda storage, loc: storage)
    names = set(model.state_dict().keys())
    for n in list(sd.keys()): 
        if n not in names and n+'_raw' in names:
            if n+'_raw' not in sd: sd[n+'_raw'] = sd[n]
            del sd[n]
    model.load_state_dict(sd)

In [9]:
# plot metrices
def plot(train_stats, valid_stats):
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15,15))
    axs[0,0].set_title("Loss Train vs Valid")
    axs[0,0].plot(train_stats.losses, 'r', label='Train')
    axs[0,0].plot(valid_stats.losses, 'g', label='Valid')
    axs[0,0].set_ylabel("Loss")
    axs[0,0].set_xlabel("Epochs")
    axs[0,0].set_xticks([i for i in range(0, len(train_stats.losses))]) 
    axs[0,0].legend()

    axs[0,1].set_title("Accuracy Train vs Valid")
    axs[0,1].plot(train_stats.accs, 'r', label='Train')
    axs[0,1].plot(valid_stats.accs, 'g', label='Valid')
    axs[0,1].set_ylabel("Accuracy")
    axs[0,1].set_xlabel("Epochs")
    axs[0,1].set_xticks([i for i in range(0, len(train_stats.accs))])
    axs[0,1].legend()

    axs[1,0].set_title("Precision vs Recall")
    axs[1,0].plot(train_stats.recs, train_stats.precs, 'r', label='Train')
    axs[1,0].plot(valid_stats.recs, valid_stats.precs, 'g', label='Valid')
    axs[1,0].set_ylabel("Precision")
    axs[1,0].set_xlabel("Recall")
    axs[1,0].legend()

In [10]:
# Simple Dataset
class SampleDataset(Dataset):
    def __init__(self, data, label=None):
        self.data = data
        if label is not None:
            self.label = label
        else:
            self.label = torch.Tensor([-1 for _ in range(len(self.data))])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

In [11]:
# train
def train(loader, model, crit, optimizer, device='cpu'):
    model.train()
    t = tqdm.tqdm(loader, leave=False, total=len(loader))
    losses = []
    ops = []
    targs = []

    for i, (ips, tgts) in enumerate(t):
        ips, tgts = ips.reshape(-1, 28*28).to(device), tgts.to(device)
        optimizer.zero_grad()
        outputs = model(ips)
        loss = crit(outputs, tgts)
        losses.append(loss.item())
        loss.backward()

        optimizer.step()

        op = torch.max(outputs.data, 1)[1]
        ops.extend(op.cpu().detach().numpy().tolist())
        targs.extend(tgts.cpu().detach().numpy().tolist())

    return losses, ops, targs, model

In [12]:
# test
def test(loader, model, crit, device):
    model.eval()
    t = tqdm.tqdm(loader, leave=False, total=len(loader))
    losses = []
    ops = []
    targs = []

    with torch.no_grad():
        for i, (ips, tgts) in enumerate(t):
            ips = ips.reshape(-1, 28*28).to(device)
            outputs = model(ips)
            op = torch.max(outputs.data, 1)[1]
            
            if not torch.all(torch.eq(tgts, -1)):
                tgts = tgts.to(device)
                loss = crit(outputs, tgts)
                losses.append(loss)
                targs.extend(tgts.cpu().detach().numpy().tolist())
                
            ops.extend(op.cpu().detach().numpy().tolist())

    return losses, ops, targs

In [13]:
def run(train_dl, valid_dl, model1, loss, optim, device, train_stat, valid_stat, model_fp, epochs=10):
    print("\nEpoch\tTrain Acc\tTrain Loss\tTrain F1\tValid Acc\tValid Loss\tValid F1")
    best_acc = 0.0
    for i in range(epochs):
        start = time.time()
        losses, ops, targs, model1 = train(train_dl, model1, loss, optim, device)
        duration = time.time() - start
        train_acc = accuracy_score(targs, ops)
        train_f1_score = f1_score(targs, ops, average='weighted')
        train_loss = sum(losses)/len(losses)
        train_prec = precision_score(targs, ops, average='weighted')
        train_rec = recall_score(targs, ops, average='weighted')
        train_stat.append(train_loss, train_acc, train_f1_score, train_prec, train_rec, duration)
        start = time.time()
        lossesv, opsv, targsv = test(valid_dl, model1, loss, device)
        duration = time.time() - start
        valid_acc = accuracy_score(targsv, opsv)
        valid_f1_score = f1_score(targsv, opsv, average='weighted')
        valid_loss = sum(lossesv)/len(lossesv)
        valid_prec = precision_score(targsv, opsv, average='weighted')
        valid_rec = recall_score(targsv, opsv, average='weighted')
        valid_stat.append(valid_loss, valid_acc, valid_f1_score, valid_prec, valid_rec, duration)

        if valid_acc > best_acc:
            best_acc = valid_acc
            save_checkpoint(model1, model_fp)

        print("\n{}\t{:06.8f}\t{:06.8f}\t{:06.8f}\t{:06.8f}\t{:06.8f}\t{:06.8f}".format(i+1, train_acc*100, train_loss, 
                                                    train_f1_score*100, valid_acc*100, 
                                                    valid_loss, valid_f1_score*100))

# Seed

In [14]:
SEED = 17

In [15]:
set_seed(SEED)

# Download MNIST Dataset and Create Dataloader

In [16]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor(),
                                          download=True)

In [17]:
bs = 128

In [18]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=bs, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=bs, 
                                          shuffle=False)

# Baseline with Cross-entropy Loss

## Model, Optimizer, Loss

In [19]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 20
lr = 1

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
model1  = MyModel(input_size, hidden_size, num_classes)

for param in model1.parameters():
    param.requires_grad = True

model1 = model1.to(device)

In [22]:
optim = torch.optim.SGD(model1.parameters(), lr=lr)

In [23]:
# Let's first use cross-entropy to get baseline performance.
ce = nn.CrossEntropyLoss()

In [24]:
train_stat = AvgStats()
valid_stat = AvgStats()

In [25]:
model_fp = './model/best_model_ce.pth'

## Train and Test

In [26]:
run(train_loader, test_loader, model1, ce, optim, device, train_stat, valid_stat, model_fp, num_epochs)


Epoch	Train Acc	Train Loss	Train F1	Valid Acc	Valid Loss	Valid F1


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


1	90.54000000	0.30405284	90.52559274	96.15000000	0.12134705	96.14853357


  0%|          | 0/469 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]


19	100.00000000	0.00123747	100.00000000	98.40000000	0.06044743	98.40046216


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


20	100.00000000	0.00111288	100.00000000	98.40000000	0.06181171	98.40051784


# PolyLoss

In [27]:
model2  = MyModel(input_size, hidden_size, num_classes)

for param in model2.parameters():
    param.requires_grad = True

model2 = model2.to(device)

In [28]:
optim = torch.optim.SGD(model2.parameters(), lr=lr)

In [29]:
pl = PolyLoss(epsilon=1)

In [30]:
train1_stat = AvgStats()
valid1_stat = AvgStats()

In [31]:
model_fp = './model/best_model_pl.pth'

In [32]:
run(train_loader, test_loader, model2, pl, optim, device, train1_stat, valid1_stat, model_fp, num_epochs)


Epoch	Train Acc	Train Loss	Train F1	Valid Acc	Valid Loss	Valid F1


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


1	88.62166667	0.52651824	88.59982791	95.91000000	0.18547447	95.91868802


  0%|          | 0/469 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]


19	99.99166667	0.00140790	99.99166655	98.12000000	0.10554124	98.12073101


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


20	100.00000000	0.00104349	100.00000000	98.09000000	0.10621919	98.09061863


# FocalLoss

In [42]:
model3  = MyModel(input_size, hidden_size, num_classes)

for param in model3.parameters():
    param.requires_grad = True

model3 = model3.to(device)

In [43]:
optim = torch.optim.SGD(model3.parameters(), lr=lr)

In [44]:
fl = PolyFocalLoss(epsilon=0.0, gamma=2.0)

In [45]:
train2_stat = AvgStats()
valid2_stat = AvgStats()

In [46]:
model_fp = './model/best_model_fl.pth'

In [47]:
run(train_loader, test_loader, model3, fl, optim, device, train2_stat, valid2_stat, model_fp, num_epochs)


Epoch	Train Acc	Train Loss	Train F1	Valid Acc	Valid Loss	Valid F1


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


1	90.08333333	0.18485392	90.08666350	95.03000000	0.07389393	95.09246005


  0%|          | 0/469 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]


19	99.99833333	0.00039638	99.99833333	98.20000000	0.02845395	98.20003542


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


20	100.00000000	0.00035775	100.00000000	98.29000000	0.02831989	98.29016653


# FocalPolyLoss

In [55]:
model4 = MyModel(input_size, hidden_size, num_classes)

for param in model4.parameters():
    param.requires_grad = True

model4 = model4.to(device)

In [56]:
optim = torch.optim.SGD(model4.parameters(), lr=lr)

In [57]:
pfl = PolyFocalLoss(epsilon=-1, gamma=2.0)

In [58]:
train3_stat = AvgStats()
valid3_stat = AvgStats()

In [59]:
model_fp = './model/best_model_pfl.pth'

In [60]:
run(train_loader, test_loader, model4, pfl, optim, device, train3_stat, valid3_stat, model_fp, num_epochs)


Epoch	Train Acc	Train Loss	Train F1	Valid Acc	Valid Loss	Valid F1


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


1	90.99333333	0.08573700	90.99287210	95.58000000	0.03271522	95.57055154


  0%|          | 0/469 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]


19	99.98500000	0.00040404	99.98500030	98.12000000	0.01381503	98.11994206


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]


20	99.99166667	0.00036728	99.99166706	98.15000000	0.01396328	98.15007985
